<a href="https://colab.research.google.com/github/elephant-xyz/notebook/blob/main/PhotoMedtaData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🐘 Welcome to Step 4 of Elephant Mining

Congratulations on reaching **Step 4**! By now, you’ve successfully **minted your County Data Group**. In this notebook, you'll use your **seed data** and **property images** to mint your **Photo Data Group**.

---

## 🧠 What You’ll Do in This Step

This notebook allows you to:

- Upload your property images  
- Mint a new **Photo Data Group**  
- Automatically generate a **fact sheet** based on the image metadata  

This step completes the visual layer of your dataset, setting you up for further data enrichment.

---

## ✅ Prerequisites

Before continuing, make sure you’ve completed the following two notebooks:

1. [📗 Notebook 1: Seed Minting](https://colab.research.google.com/drive/14tSNSP8Pe-mY4VwX9JhXgfyOvzmN3kC0?usp=chrome_ntp)  
2. [📘 Notebook 2: County Data Minting](https://colab.research.google.com/drive/1ZI_eScKFh2kDIZgwXljhOgBIgrenDhRi?usp=chrome_ntp)

After running both, you should have the following output files ready:

- `upload-results.json`  
- `submit.zip`

Also ensure you have:

- **OpenAI API Key**: Valid API key for AI processing capabilities
- **AWS Account with Credentials**: AWS Access Key ID and Secret Access Key for cloud services
- **Pinata Account**: JWT token for IPFS storage services
- **Elephant Address**: Private key for blockchain integration

**Required Environment Variables:**
---

## 📸 In This Notebook

Once your image files are uploaded:

1. The images will be minted into the **Photo Data Group**  
2. A **fact sheet** will be generated for inspection  
3. You can continue with **image-based metadata extraction**  
4. This will lead to a complete and enriched data product  

---



## 📥 Step 1: Upload the `.env` File

This notebook requires a `.env` file containing your API keys and credentials. Create a file with the following environment variables:

| Variable Name | Purpose |
|---|---|
| `OPENAI_API_KEY` | Access to OpenAI API |
| `AWS_ACCESS_KEY_ID` | AWS access key |
| `AWS_SECRET_ACCESS_KEY` | AWS secret access key |
| `S3_BUCKET_NAME` | Your S3 bucket name |
| `IMAGES_DIR` | Directory path for images |
| `ELEPHANT_PRIVATE_KEY` | Elephant wallet private key |
| `PINATA_JWT` | Pinata authentication token |

### To upload:
1. Click the **folder icon** 📂 in the left sidebar
2. Click the **"Upload"** button
3. Select your `.env` file

### Example `.env` file:
```env
OPENAI_API_KEY=sk-XXXXXXXXXXXXXXXXXXXXXXXXXXXX
AWS_ACCESS_KEY_ID=XXXXXX
AWS_SECRET_ACCESS_KEY=XXXXXX
S3_BUCKET_NAME=your-s3-bucket-name-here
IMAGES_DIR=images
ELEPHANT_PRIVATE_KEY=xxxxx
PINATA_JWT=xxxxx
```

> ⚠️ **Security Note:** Never commit your `.env` file to version control or share it publicly.


## Step 2: Upload `upload_results.csv`

Upload the `upload_results.csv` file to the `/content/` directory.

> 📌 **Important**: This file was generated by running **Step 2** of the [Seed Data Notebook](https://colab.research.google.com/drive/14tSNSP8Pe-mY4VwX9JhXgfyOvzmN3kC0?usp=sharing#scrollTo=OFKp4E49651Z)

The file should now be downloaded and ready to upload to `/content/upload_results.csv`

## Step 3: Upload `submit.zip`

Upload the `submit.zip` file to the `/content/` directory.

> 📌 **Important**: This file was generated by running **Step 3** of the [County Data Notebook](https://colab.research.google.com/drive/1ZI_eScKFh2kDIZgwXljhOgBIgrenDhRi#scrollTo=HA0ppLFpUm1j)

The file should now be downloaded and ready to upload to `/content/submit.zip`

## Step 4: Verify Data Exists

Once both files are uploaded to `/content/`, you can proceed with the main workflow that depends on these generated datasets.

**Expected file locations:**
- `/content/upload-results.csv`
- `/content/submit.zip`







In [10]:
!ls -la /content/upload-results.csv
!ls -la /content/submit.zip

-rw-r--r-- 1 root root 449 Jul 25 06:17 /content/upload-results.csv
-rw-r--r-- 1 root root 42731 Jul 25 06:17 /content/submit.zip


##Step 5: Install Package & Setup Folders
This step:

Installs the photo-meta-data-ai package from GitHub
Creates all necessary folders for the project
Saves installation details to a log file for troubleshooting

Takes 1-2 minutes to complete. Once finished, you'll have the AI package installed and folder structure ready for processing photos.



In [ ]:
# 1. Install the package
!pip install --force-reinstall --no-cache-dir git+https://github.com/elephant-xyz/photo-meta-data-ai.git > /content/install_log.txt 2>&1

# 2. Set up folders
!colab-folder-setup


##Step 6: Upload Images to Property Subfolders
Upload your property images into the pre-created subfolders:

Each property already has a subfolder named with its Parcel ID under the images folder
Simply drag and drop your images into the correct property subfolder
All images for a specific property should go in that property's designated folder

The AI will process each property's images and generate metadata organized by Parcel ID.






## Step 7 Uploading Photos

In [3]:
!process-photo-data
!npx -y @elephant-xyz/cli@latest validate-and-upload photo_data_group --output-csv photos.csv

  - Processed bafkreigzz5foh5ts76vvhxphzulptpnjwznog6lcnxw5wsvfqa7zlxeioa: 74 images, root.json created

Processing complete:
  - Processed 1 directories with images
  - Generated 74 image metadata files
  - Each directory has its own root.json
  - Supported formats: .jpeg, .jpg, .png, .webp
⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸🐘 Elephant Network CLI - Validate and Upload

7Initializing    |████████████████████████████████████████| 100% | 0/0 | Errors: 0 | Skipped: 0 | 0s | ETA: 0sProcessing Files |░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░| 0% | 0/1 | Errors: 0 | Skipped: 0 | 0s | ETA: NFsProcessing Files |░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░| 0% | 0/1 | Errors: 0 | Skipped: 0 | 1s | ETA: NFsProcessing Files |░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░| 0% | 0/1 | Errors: 0 | Skipped: 0 | 2s | ETA: NFsProcessing Files |░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░| 0% | 0/1 |

## Step 8: Submitting Your Data to the Blockchain

### Submitting Your Data

After running the upload command in the notebook:

1. **Download your results file**
   - The notebook will generate `photos.csv`
   - This file contains your data hashes and IPFS CIDs
   - Download it to your computer

2. **Visit the Oracle Submission Portal**
   - Go to https://oracle.elephant.xyz/
   - Connect your MetaMask wallet when prompted
   - Upload your `submit-results.csv` file

3. **Submit transactions**
   - The portal will read your CSV and prepare transactions
   - Click "Submit to Contract" to begin
   - MetaMask will pop up for each data entry
   - Confirm each transaction (small gas fee applies)
   - Wait for confirmations between submissions

Once complete, your data is permanently recorded on the blockchain. You'll receive vMahout tokens as rewards after consensus is reached (when 3 different oracles submit matching data hashes).

## Step 9: Setup and Run AWS Rekognition

The system automatically sets up and runs Amazon Rekognition to analyze your property images:

- Connects to AWS Rekognition service for AI-powered image analysis
- Processes all images in your property folders automatically
- Extracts detailed information like room types, architectural features, and property characteristics

No action needed from you - the system handles everything automatically and will notify you when processing is complete.


In [4]:
!bucket-manager
!unzip-county-data
!upload-to-s3
!photo-categorizer


📊 COMPREHENSIVE CATEGORIZATION SUMMARY

🏠 TOTAL PROPERTIES PROCESSED: 2
🖼️  TOTAL IMAGES: 196
✅ TOTAL CATEGORIZED: 196
📈 SUCCESS RATE: 100.0%

📁 OVERALL CATEGORY BREAKDOWN:
   exterior: 68 images
   kitchen: 42 images
   living_room: 38 images
   bedroom: 20 images
   other: 12 images
   closet: 6 images
   garage: 4 images
   bathroom: 2 images
   laundry: 2 images
   pool: 2 images

🏠 PROPERTY-BY-PROPERTY BREAKDOWN:
--------------------------------------------------------------------------------

📍 Property: 30434108090030050
   Address: Property 30434108090030050
   Total Images: 48
   Categorized: 48
   Success Rate: 100.0%
   Categories:
     • kitchen: 18 images
     • living_room: 10 images
     • exterior: 8 images
     • bedroom: 8 images
     • bathroom: 2 images
     • closet: 2 images

📍 Property: 52434205310037080
   Address: Property 52434205310037080
   Total Images: 148
   Categorized: 148
   Success Rate: 100.0%
   Categories:
     • exterior: 60 images
     • living_

## Step 10: Running AI to Extract Data from Images

The AI system now analyzes your property images to extract valuable metadata:

1. **Image Analysis**: AI examines each photo to identify rooms, features, and property details
2. **Data Extraction**: System pulls out structured information like room types, square footage estimates, architectural elements, and condition assessments

The process runs automatically across all your uploaded property images, generating comprehensive metadata reports for each parcel.



In [6]:
!ai-analyzer --local-folders --parallel-categories --all-properties
!property-summarizer --all-properties


PROPERTY SUMMARY: 52434205310037080

📋 LAYOUTS (11 total)
----------------------------------------
Space Types:
  • Dining Room
  • Closet
  • Bedroom
  • Full Bathroom
  • Home Office
  • Living Room
  • Laundry Room
  • Kitchen
  • Attached Garage
  • Dining Room: 
  • Closet: 
  • Bedroom: 
  • Full Bathroom: 
  • Home Office: 
  • Living Room: 
  • Laundry Room: 
  • Kitchen: 
  • Full Bathroom: 
  • Attached Garage: 
  • Bedroom: 

🏠 STRUCTURE
----------------------------------------
  No structure data found

🌳 LOT
----------------------------------------
  No lot data found

⚡ UTILITIES
----------------------------------------
  No utility data found

🔌 APPLIANCES (6 total)
----------------------------------------
Types: Washing Machine, Microwave, Refrigerator, Toaster Oven, Oven
  • Washing Machine
  • Microwave
  • Refrigerator
  • Toaster Oven
  • Oven


🎉 COMPLETED: Summarized 1 properties
  • 52434205310037080: 11 layouts, 6 appliances


## Step 11: Data Validation and Submission

The system validates extracted data and prepares it for final submission:

1. **Data Validation**: Reviews and verifies all extracted metadata for accuracy
2. **Submission Preparation**: Validated data is formatted and organized for CLI submission
3. **CLI Submission**: System automatically submits the processed data through the command line interface
4. **Fact Sheet Generation**: Creates comprehensive property fact sheets with all extracted information, images, and metadata

Final deliverables include validated property reports and detailed fact sheets ready for use.


In [7]:
!fix-schema-validation
!copy-all-data-for-submission
!copy-all-files-from-zip
!npx @elephant-xyz/cli@latest validate-and-upload submit-photo --output-csv submit-results.csv

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙🐘 Elephant Network CLI - Validate and Upload

7Initializing    |████████████████████████████████████████| 100% | 0/0 | Errors: 0 | Skipped: 0 | 0s | ETA: 0s7Fetching Schemas |░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░| 0% | 0/2 | Errors: 0 | Skipped: 0 | 0s | ETA: 0sPre-fetching Schemas |░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░| 0% | 0/1 | Errors: 0 | Skipped: 0 | 0s | ETA: NFsFetching Schemas |████████████████████░░░░░░░░░░░░░░░░░░░░| 50% | 1/2 | Errors: 0 | Skipped: 0 | 0s | ETA: 1sFetching Schemas |████████████████████████████████████████| 100% | 2/2 | Errors: 0 | Skipped: 0 | 0s | ETA: 0s8
Processing Files |░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░| 0% | 0/2 | Errors: 0 | Skipped: 0 | 0s | ETA: NFsProcessing Files |░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░| 0% | 0/2 | Errors: 0 | Skipped: 0 | 1s | ETA: NFsProcessing Files |░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░| 0% | 0/2 | Errors: 0 | Skipped: 0 | 2s | ETA: NFsProcessing Files |░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░

## Step 12: Submitting Your Data to the Blockchain

### Submitting Your Data

After running the upload command in the notebook:

1. **Download your results file**
   - The notebook will generate `submit-results.csv`
   - This file contains your data hashes and IPFS CIDs
   - Download it to your computer

2. **Visit the Oracle Submission Portal**
   - Go to https://oracle.elephant.xyz/
   - Connect your MetaMask wallet when prompted
   - Upload your `submit-results.csv` file

3. **Submit transactions**
   - The portal will read your CSV and prepare transactions
   - Click "Submit to Contract" to begin
   - MetaMask will pop up for each data entry
   - Confirm each transaction (small gas fee applies)
   - Wait for confirmations between submissions

Once complete, your data is permanently recorded on the blockchain. You'll receive vMahout tokens as rewards after consensus is reached (when 3 different oracles submit matching data hashes).

## Step 13: Package and Download Results

This step creates downloadable files with all your processed data. The system will:

1. **Create Download Package**: Automatically zip the submit-photos folder containing all fact sheets and processed images

2. **Download Results**: Two files will be made available for download

**Files to Download:**
- `submit-results.csv` - Structured data with all extracted property metadata
- `submit-photo.zip` - Complete package containing fact sheets and processed images

Your processed property data is now saved locally for use.

In [8]:
!zip -r submit-photo.zip submit-photo/ > /dev/null 2>&1



## Step 14: Cleanup

Final and optional step to save your results and clean up the workspace:

3. **Cleanup Workspace**: After downloading, the system removes all temporary files and folders including:
  - `images` folder (uploaded property photos)
  - `output` folder (processing files)
  - `county-data` folder (temporary data)
  - `submit-photos` folder (final results)
  - `logs` folder (processing logs)

**Important**: Make sure to download your results before the cleanup step, as all files will be permanently deleted from the workspace.

In [9]:
!rm -rf images/ output/ county-data/ submit-photo/ logs/ > /dev/null 2>&1
